In [1]:
import random
import math

def generate_vrppd_instance(seed, grid_km, num_requests):
    if seed is not None:
        random.seed(seed)
    
    data = {}
    
    # Depot at the center
    depot = (grid_km / 2, grid_km / 2)
    locations = [depot]

    # Generate pickup and delivery points
    pickups_deliveries = []
    for _ in range(num_requests):
        pickup = (random.uniform(0, grid_km), random.uniform(0, grid_km))
        delivery = (random.uniform(0, grid_km), random.uniform(0, grid_km))
        locations.append(pickup)
        locations.append(delivery)
        pickups_deliveries.append([len(locations) - 2, len(locations) - 1])

    # Compute Euclidean distance matrix
    def euclidean(p1, p2):
        return int(round(math.hypot(p1[0] - p2[0], p1[1] - p2[1])))

    distance_matrix = [
        [euclidean(loc1, loc2) for loc2 in locations]
        for loc1 in locations
    ]

    data['nodes'] = [{'id': idx, 'x': round(coord[0], 2), 'y': round(coord[1], 2)} for idx, coord in enumerate(locations)]
    data['distance_matrix'] = distance_matrix
    data['depot'] = 0
    data['pickups_deliveries'] = pickups_deliveries

    return data

seed = 1234;
grid_km = 10;
num_requests = 10;
data = generate_vrppd_instance(seed, grid_km, num_requests)
data['num_vehicles'] = 1
data['max_travel_distance_vehicle'] = 1_000_000
data['grid_km'] = grid_km
data['num_requests'] = num_requests
data['seed'] = seed
print(data)

{'nodes': [{'id': 0, 'x': 5.0, 'y': 5.0}, {'id': 1, 'x': 9.66, 'y': 4.41}, {'id': 2, 'x': 0.07, 'y': 9.11}, {'id': 3, 'x': 9.39, 'y': 5.82}, {'id': 4, 'x': 6.72, 'y': 0.84}, {'id': 5, 'x': 7.66, 'y': 2.37}, {'id': 6, 'x': 0.31, 'y': 7.89}, {'id': 7, 'x': 3.46, 'y': 6.23}, {'id': 8, 'x': 6.16, 'y': 1.49}, {'id': 9, 'x': 1.83, 'y': 1.14}, {'id': 10, 'x': 0.15, 'y': 4.87}, {'id': 11, 'x': 9.65, 'y': 0.65}, {'id': 12, 'x': 5.41, 'y': 4.66}, {'id': 13, 'x': 6.01, 'y': 0.89}, {'id': 14, 'x': 5.79, 'y': 2.7}, {'id': 15, 'x': 5.56, 'y': 6.45}, {'id': 16, 'x': 4.81, 'y': 3.55}, {'id': 17, 'x': 2.49, 'y': 9.34}, {'id': 18, 'x': 4.53, 'y': 5.3}, {'id': 19, 'x': 0.19, 'y': 5.08}, {'id': 20, 'x': 0.06, 'y': 1.44}], 'distance_matrix': [[0, 5, 6, 4, 5, 4, 6, 2, 4, 5, 5, 6, 1, 4, 2, 2, 1, 5, 1, 5, 6], [5, 0, 11, 1, 5, 3, 10, 6, 5, 8, 10, 4, 4, 5, 4, 5, 5, 9, 5, 9, 10], [6, 11, 0, 10, 11, 10, 1, 4, 10, 8, 4, 13, 7, 10, 9, 6, 7, 2, 6, 4, 8], [4, 1, 10, 0, 6, 4, 9, 6, 5, 9, 9, 5, 4, 6, 5, 4, 5, 8, 5, 9, 

In [2]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import time

# ---------------------------
# Create routing model
# ---------------------------
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']), data['num_vehicles'], data['depot']
)
routing = pywrapcp.RoutingModel(manager)

# ---------------------------
# Define distance callback
# ---------------------------
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# ---------------------------
# Add Distance dimension
# ---------------------------
routing.AddDimension(
    transit_callback_index,
    0,      # no waiting time at nodes
    data['max_travel_distance_vehicle'],   # max distance per vehicle
    True,   # start cumul to zero
    "Distance"
)
distance_dimension = routing.GetDimensionOrDie("Distance")

# ---------------------------
# Add Pickup and Delivery constraints
# ---------------------------
for pickup, delivery in data['pickups_deliveries']:
    pickup_index = manager.NodeToIndex(pickup)
    delivery_index = manager.NodeToIndex(delivery)
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(
        routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
    )
    routing.solver().Add(
        distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index)
    )

print("Instance loaded")

# ---------------------------
# Search parameters
# ---------------------------
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
#search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_parameters.time_limit.seconds = 30
search_parameters.log_search = True  # Enable solver logs

# ---------------------------
# Solve and print solution
# ---------------------------

# Measure solving time manually
start_time = time.time()
solution = routing.SolveWithParameters(search_parameters)
end_time = time.time()

print(f"Solving time: {end_time - start_time:.2f} seconds")

if solution:
    print("Model solved.")
else:
    print("No solution found.")

Instance loaded
Solving time: 0.02 seconds
Model solved.


In [3]:
import matplotlib.pyplot as plt
import random
from IPython.display import display
import ipywidgets as widgets
import matplotlib.patches as mpatches

node_coords = {node["id"]: (node["x"], node["y"]) for node in data["nodes"]}

# Store route text for Markdown display
def get_solution_text(manager, routing, solution):
    total_distance = 0
    output = "\n\n\n"
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"### Route for vehicle {vehicle_id}:\n\n"
        route_distance = 0
        route_nodes = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route_nodes.append(node)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route_nodes.append(manager.IndexToNode(index))
        plan_output += " → ".join(str(n) for n in route_nodes) + "\n"
        plan_output += f"**Distance of the route:** {route_distance}m\n\n"
        output += plan_output
        total_distance += route_distance
    output += f"## Total distance of all routes: {total_distance}m\n"
    return output

# Calcola offset verticale in pixel e converti in unità dati
def offset_in_data_units(grid_km, fraction=0.015):
    """Calcola offset verticale in unità dati proporzionale alla griglia."""
    return grid_km * fraction
    
# Plotting function (returns an Axes object)
def plot_solution(ax, manager, routing, solution, data, node_coords):
    # Base colors for vehicles with alpha for lighter lines
    vehicle_colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']
    vehicle_alpha = 0.3

    pickup_color = 'orange'
    delivery_color = 'purple'
    depot_color = 'red'

    # Map nodes to their pickup-delivery pairs
    pickup_delivery_map = {}
    for pickup, delivery in data["pickups_deliveries"]:
        pickup_delivery_map[pickup] = (pickup, delivery)
        pickup_delivery_map[delivery] = (pickup, delivery)

    offset_pickup = offset_in_data_units(data['grid_km'], 0.037)  # 10 pixel sopra/sotto la label
    offset_delivery = offset_in_data_units(data['grid_km'], 0.015)  # 10 pixel sopra/sotto la label

    # Plot nodes with different shapes/colors
    for node, (x, y) in node_coords.items():
        if node == data["depot"]:
            ax.plot(x, y, 's', color=depot_color, markersize=10, label='Depot')
            ax.text(x + 1, y + 1, f"{node}", fontsize=9, fontweight='bold')
            continue

        if node in pickup_delivery_map:
            pickup_id, delivery_id = pickup_delivery_map[node]
            slack = 0;
            if node == pickup_id:
                # Pickup node: orange, triangle up, bold pickup id
                ax.plot(x, y+offset_pickup, '^', color=pickup_color, markersize=10)
                label = f"$\\bf{{{pickup_id}}}$-{delivery_id}"
            else:
                # Delivery node: purple, triangle down, bold delivery id
                ax.plot(x, y-offset_delivery, 'v', color=delivery_color, markersize=10)
                label = f"{pickup_id}-$\\bf{{{delivery_id}}}$"
            
            ax.text(
                x, y,  # shift label 2 units above node; adjust as needed
                label,
                fontsize=9,
                ha='center',    # center horizontally
                va='bottom',    # align bottom of text at y+2, so text goes upwards
                bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2')
            )


        else:
            # Other nodes: black circle
            ax.plot(x, y, 'o', color='black', markersize=6)
            ax.text(x + 1, y + 1, str(node), fontsize=9)

    # Plot vehicle routes with lighter colors
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node_coords[node])
            index = solution.Value(routing.NextVar(index))
        route.append(node_coords[manager.IndexToNode(index)])
        xs, ys = zip(*route)
        ax.plot(xs, ys, color=vehicle_colors[vehicle_id % len(vehicle_colors)], 
                linewidth=2, alpha=vehicle_alpha, label=f'Vehicle {vehicle_id}')

    ax.set_title("Vehicle Routes")

    # Create custom legend handles for depot, pickups, deliveries, and routes
    depot_handle = mpatches.Patch(color=depot_color, label='Depot')
    pickup_handle = plt.Line2D([], [], color=pickup_color, marker='^', linestyle='None', markersize=10, label='Pickup points')
    delivery_handle = plt.Line2D([], [], color=delivery_color, marker='v', linestyle='None', markersize=10, label='Delivery points')
    vehicle_handles = [
        plt.Line2D([], [], color=vehicle_colors[i % len(vehicle_colors)], lw=2, alpha=vehicle_alpha, label=f'Vehicle {i}')
        for i in range(data["num_vehicles"])
    ]

    handles = [depot_handle, pickup_handle, delivery_handle] + vehicle_handles
    ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5), title="Legend")
    ax.grid(True)

# Display side-by-side using widgets
if solution:
    text_output = get_solution_text(manager, routing, solution)
    out_text = widgets.Output()
    out_plot = widgets.Output()

    with out_text:
        print(text_output)

    with out_plot:
        fig, ax = plt.subplots(figsize=(8, 8))
        plot_solution(ax, manager, routing, solution, data, node_coords)
        plt.show()

    display(widgets.HBox([out_text, out_plot]))
else:
    print("No solution found.")